In [1]:
import pandas as pd
import re
import random
import os
import codecs
import itertools

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import warnings
from konlpy.tag import Kkma; tokenizer = Kkma()

In [2]:
warnings.filterwarnings("ignore", category=UserWarning)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
MAX_LENGTH = 20

In [4]:
# 기본 단어 토큰 값
SOS_token = 0 # 문장의 시작(SOS, Start Of Sentence)을 나타내는 토큰
EOS_token = 1 # 문장의 끝(EOS, End Of Sentence)을 나태는 토큰

class Voc :
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.index2word = {}
        self.word2count = {0: "SOS", 1: "EOS"}
        self.n_words = 3 # SOS, EOS 2개
        
    def addSentence(self, sentence):
        #for word in tokenizer.morphs(sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [5]:
def normalizeString(s):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣 ^☆; ^a-zA-Z.!?;0-9]+')
    result = hangul.sub('', s)
    return result

In [6]:
def readText():
    print("Reading lines...")
    
    inputs = open('./data/normal chatbot data/source.txt', encoding='utf-8').read().strip().split('\n')
    outputs = open('./data/normal chatbot data/target.txt', encoding='utf-8').read().strip().split('\n')
 
    inputs = [normalizeString(s) for s in inputs]
    outputs = [normalizeString(s) for s in outputs]
    print(len(inputs))
    print(len(outputs))
    
    inp = Voc('input')
    outp = Voc('output')
    
    pair = []
    for i in range(len(inputs)):
        pair.append([inputs[i], outputs[i]])
    return inp, outp, pair

In [7]:
def prepareData():
    input_voc, output_voc, pairs = readText()
    print("Read %s sentence pairs" % len(pairs))
 
    print("Counting words...")
    for pair in pairs:
        input_voc.addSentence(pair[0])
        output_voc.addSentence(pair[1])
    print("Counted words:")
    print(input_voc.name, input_voc.n_words)
    print(output_voc.name, output_voc.n_words)
    return input_voc, output_voc, pairs
 
input_voc, output_voc, pairs = prepareData()
print(random.choice(pairs))

Reading lines...
11823
11823
Read 11823 sentence pairs
Counting words...
Counted words:
input 14260
output 9993
['책 선물이 제일 나을 듯', '어떤 분야를 좋아하는지 알아보세요.']


In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [9]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [10]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [11]:
def indexesFromSentence(voc, sentence):
    #return [voc.word2index[word] for word in tokenizer.morphs(sentence)]
    return [voc.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(voc, sentence):
    indexes = indexesFromSentence(voc, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_voc, pair[0])
    target_tensor = tensorFromSentence(output_voc, pair[1])
    return (input_tensor, target_tensor)

In [12]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing 포함: 목표를 다음 입력으로 전달
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Teacher forcing 미포함: 자신의 예측을 다음 입력으로 사용
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # 입력으로 사용할 부분을 히스토리에서 분리

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [13]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [14]:
def trainIters(encoder, decoder, n_iters, print_every=100, plot_every=10, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # print_every 마다 초기화
    plot_loss_total = 0  # plot_every 마다 초기화

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [15]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # 주기적인 간격에 이 locator가 tick을 설정
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [16]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_voc, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_voc.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [17]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [29]:
def evaluateInput(encoder, attn_decoder):
    input_sentence = ''
    while(1):
        try:
            input_sentence = input('USER > ')
            if input_sentence == 'q' or input_sentence == 'quit': break
            output_words, attentions = evaluate(encoder, attn_decoder, input_sentence)
            output_words[:] = [x for x in output_words if not (x == 'EOS')]
            print('BOT  >', ' '.join(output_words))
            
        except KeyError:
            print("Error")

In [19]:
hidden_size = 256
encoder = EncoderRNN(input_voc.n_words, hidden_size).to(device)
attn_decoder = AttnDecoderRNN(hidden_size, output_voc.n_words, dropout_p=0.1).to(device)

In [20]:
trainIters(encoder, attn_decoder, 2000)

0m 2s (- 0m 54s) (100 5%) 5.7866
0m 4s (- 0m 42s) (200 10%) 5.1376
0m 6s (- 0m 37s) (300 15%) 4.9218
0m 8s (- 0m 33s) (400 20%) 4.7139
0m 10s (- 0m 30s) (500 25%) 5.6451
0m 12s (- 0m 28s) (600 30%) 5.6222
0m 14s (- 0m 26s) (700 35%) 5.6952
0m 16s (- 0m 24s) (800 40%) 5.7391
0m 18s (- 0m 22s) (900 45%) 5.7163
0m 20s (- 0m 20s) (1000 50%) 5.4061
0m 22s (- 0m 18s) (1100 55%) 5.5064
0m 24s (- 0m 16s) (1200 60%) 5.5388
0m 26s (- 0m 14s) (1300 65%) 5.6013
0m 28s (- 0m 12s) (1400 70%) 5.4235
0m 30s (- 0m 10s) (1500 75%) 5.2771
0m 32s (- 0m 8s) (1600 80%) 5.6777
0m 34s (- 0m 6s) (1700 85%) 5.9666
0m 36s (- 0m 4s) (1800 90%) 5.5850
0m 38s (- 0m 2s) (1900 95%) 5.2694
0m 40s (- 0m 0s) (2000 100%) 5.2544


In [21]:
evaluateRandomly(encoder, attn_decoder)

> 나 좀 내버려 두면 좋겠어
= 많이 지쳤나봐요.
< 서로 <EOS>

> 지금 이감정
= 당신만이 알 수 있어요.
< 서로 <EOS>

> 10년 연애. 헤어졌습니다.
= 더 공허함이 크시겠네요.
< 서로 <EOS>

> 오늘 너무 힘들다
= 고생 많았어요.
< 서로 <EOS>

> 자기관리가 답인듯.
= 못해봤던 것들을 해보세요.
< 서로 <EOS>

> 요즘 눈에 띄는 남자애가 있는데
= 관심을 갖고 지켜보세요.
< 서로 <EOS>

> 짝남이 곧 군대를 가요.
= 마음이 착잡하겠어요.
< 서로 <EOS>

> 숙제 산더미
= 지금도 늦지 않았어요.
< 서로 <EOS>

> 식욕폭발
= 맛있게 먹으면 영칼로리!
< 서로 <EOS>

> 그리고 3년하고도 5개월
= 이젠 기억을 놔주는 건 어떨까요.
< 서로 <EOS>



In [ ]:
encoder.eval()
attn_decoder.eval()

evaluateInput(encoder, attn_decoder)

USER > 숙제 산더미
BOT  > 서로 <EOS>
USER > 재미없다.
BOT  > 서로 <EOS>
USER > 서로?
Error
USER > 이름이 뭐야?
BOT  > 서로 <EOS>
